## About this notebook

*[Jigsaw Multilingual Toxic Comment Classification](https://www.kaggle.com/c/jigsaw-multilingual-toxic-comment-classification)* is the 3rd annual competition organized by the Jigsaw team. It follows *[Toxic Comment Classification Challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge)*, the original 2018 competition, and *[Jigsaw Unintended Bias in Toxicity Classification](https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification)*, which required the competitors to consider biased ML predictions in their new models. This year, the goal is to use english only training data to run toxicity predictions on many different languages, which can be done using multilingual models, and speed up using TPUs.

Many awesome notebooks has already been made so far. Many of them used really cool technologies like [Pytorch XLA](https://www.kaggle.com/theoviel/bert-pytorch-huggingface-starter). This notebook instead aims at constructing a **fast, concise, reusable, and beginner-friendly model scaffold**. 

**THIS DOES NOT USE ANY TRANSLATED DATA, BUT IT DOES TRAIN ON THE VALIDATION SET.**


### References
* Original Author: [@xhlulu](https://www.kaggle.com/xhlulu/)
* Original notebook: [Link](https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras)

In [ ]:
# линейная алгебра
import numpy as np
# чтение файлов и обработка данных
import pandas as pd

# огромная библиотека от Google для машинного обучения https://www.tensorflow.org
import tensorflow as tf
# Dense - для организации полносвязнных слоёв, которые будем использовать для построения модели
# Input - используется для создания тензора
from tensorflow.keras.layers import Dense, Input
# Adam - метод для стохастической оптимизации https://arxiv.org/abs/1412.6980v8
from tensorflow.keras.optimizers import Adagrad
# Model - группирует слои в объект с функциями обучения и вывода
from tensorflow.keras.models import Model

# внутренняя Kaggle кухня
from kaggle_datasets import KaggleDatasets

# большой тулкит для работы с естественными языками, включает в себя множество моделей
import transformers
# TFAutoModel - для создания предобученной модели
# AutoTokenizer - предобученный токенайзер
from transformers import TFAutoModel, AutoTokenizer

# красивый progress bar
from tqdm.notebook import tqdm

## Helper Functions

In [ ]:
def regular_encode(texts, tokenizer, maxlen=512):
    """
    Вспомогательная функция для представление текста в виде вектора, который представляет набор айдишников
    слов в общем словаре с дополнительной информацией о слове
    """
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

In [ ]:
def build_model(transformer, max_len=512):
    # Вход для модели
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    
     # Выход обученной модели
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    
    # Изменяем выход под нашу задачу
    out = Dense(1, activation='sigmoid')(cls_token)
    
    # Формируем итоговую модель и компилируем ее
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adagrad(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

## TPU Configs

In [ ]:
# Инициализация оборудования для работы на TPU
try:
    # TPU detection.
    # No parameters necessary if TPU_NAME environment variable is set:
    # this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

    print(f'Running on TPU {tpu.master()}')
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print(f'REPLICAS: {strategy.num_replicas_in_sync}')

In [ ]:
# TPU кухня
AUTO = tf.data.experimental.AUTOTUNE

# конфигурация для работы с моделью:
# число эпох
EPOCHS = 2
# размер обучающей выборки, которая будет пропускаться через нейронную сеть
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
# максимальная длина твита
MAX_LEN = 192
# предобученная модель, которую мы возьмём за основу для анализа твитов
MODEL = 'jplu/tf-xlm-roberta-large'

## Create fast tokenizer

In [ ]:
# инициализируем токенайзер для модели
tokenizer = AutoTokenizer.from_pretrained(MODEL)

## Load text data into memory

In [ ]:
# загружаем первый источник с текстами и некоторыми признаками
train1 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv")
# загружаем второй источник с текстами и некоторыми признаками
train2 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv")
# нормализуем данные, округляя в большую сторону, чтобы использовать их для тренировки
train2.toxic = train2.toxic.round().astype(int)

# источник с текстами для валидации
valid = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
# источник для проверки модели
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
# файл, куда будем писать ответы
sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

In [ ]:
train1.shape

In [ ]:
train1.head()

In [ ]:
train2.shape

In [ ]:
train2.head()

In [ ]:
valid.shape

In [ ]:
valid.head()

In [ ]:
test.shape

In [ ]:
test.head()

In [ ]:
sub.shape

In [ ]:
sub.head()

In [ ]:
train1.toxic.value_counts()

In [ ]:
train2.toxic.value_counts()

Объединим тренировочные данные для обучения, при этом уберём большинство токсичных комментариев, чтобы выборка с нетоксичными комментариями не преобладала в обучении.

In [ ]:
# prev 21384
# prev 112226

train = pd.concat([
    train1[['comment_text', 'toxic']].query('toxic==1').sample(n=21384, random_state=0),
    train1[['comment_text', 'toxic']].query('toxic==0'),
    train2[['comment_text', 'toxic']].query('toxic==1').sample(n=112226, random_state=0),
    train2[['comment_text', 'toxic']].query('toxic==0')
])

Применим наш токенайзер, чтобы получить векторные представления твитов.

In [ ]:
%%time 

x_train = regular_encode(train.comment_text.values, tokenizer, maxlen=MAX_LEN)
x_valid = regular_encode(valid.comment_text.values, tokenizer, maxlen=MAX_LEN)
x_test = regular_encode(test.content.values, tokenizer, maxlen=MAX_LEN)

y_train = train.toxic.values
y_valid = valid.toxic.values

## Build datasets objects

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

## Load model into the TPU

In [ ]:
%%time
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=MAX_LEN)

model.summary()

## Train Model

Обучаем модель на тренировочном сабсете, который полностью на английском языке.

In [ ]:
n_steps = x_train.shape[0]

train_history_1 = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

Модель обучилась на огромной выборке только на английском языке, дообучим ещё на оставшейся выборке, которая гораздо меньше, но содержит смесь различных языков.

In [ ]:
n_steps = x_valid.shape[0] // BATCH_SIZE

train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS
)

## Submission

In [ ]:
sub['toxic'] = model.predict(test_dataset, verbose=1)
sub.to_csv('submission.csv', index=False)